<a href="https://colab.research.google.com/github/you-sif/you-sif/blob/main/Project_DS_2_Test_Fiber_Orange_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the dataset from an Excel file
file_path = '/content/preprocessed_data.xlsx'
df = pd.read_excel(file_path)


In [ ]:
# Define categorical and numerical columns
categorical_columns = ["GOVERNORATE", "CUSTOMER_GENDER", "MIGRATION_FLAG"]
numerical_columns = ["COMMITMENT", "OF_SPEED", "CUSTOMER_AGE_YEARS", "Customer with orange_MONTHS"]
label = "TARGET"

# Separate features (X) and target variable (y)
X = df[categorical_columns + numerical_columns]
y = df[label]

# Apply Random Oversampling and Undersampling
ros = RandomOverSampler(sampling_strategy=0.1)
rus = RandomUnderSampler(sampling_strategy=0.9)

X_resampled, y_resampled = ros.fit_resample(X, y)
X_resampled, y_resampled = rus.fit_resample(X_resampled, y_resampled)

# Encode categorical variables using Binary Encoding
encoder = ce.BinaryEncoder(cols=categorical_columns, return_df=True)
X_resampled_encoded = encoder.fit_transform(X_resampled)

# Train a logistic regression model using cross-validation
model = LogisticRegression()
scores = cross_val_score(model, X_resampled_encoded, y_resampled, cv=10)  # You can adjust the number of folds as needed

# Print cross-validation scores
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())

# Fit the model on the entire resampled dataset
model.fit(X_resampled_encoded, y_resampled)

# Generate a classification report on the test data
y_pred = model.predict(X_resampled_encoded)  # Predict on the resampled data
classification_rep = classification_report(y_resampled, y_pred)

print("Classification report on resampled data:")
print(classification_rep)

# Save the trained model to a file
model_path = 'trained_model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

# Load the pretrained model
with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

# Preprocess new data (replace this with your actual new data)
X_new = pd.DataFrame({
    "GOVERNORATE": ["Irbid"],
    "CUSTOMER_GENDER": ["M"],
    "MIGRATION_FLAG": ["y"],
    "COMMITMENT": [24],
    "OF_SPEED": [100],
    "CUSTOMER_AGE_YEARS": [37],
    "Customer with orange_MONTHS": [41]
})

# Encode categorical variables for the new data
X_new_encoded = encoder.transform(X_new)

# Make predictions on the new data
new_data_predictions = loaded_model.predict(X_new_encoded)

print("Predictions for new data:")
print(new_data_predictions)


Cross-validation scores: [0.75403226 0.74193548 0.73991935 0.72718104 0.73373676 0.72465961
 0.74483106 0.74583964 0.75592537 0.75945537]
Mean accuracy: 0.7427515941958258
Classification report on resampled data:
              precision    recall  f1-score   support

           0       0.78      0.73      0.75     10438
           1       0.72      0.77      0.74      9395

    accuracy                           0.75     19833
   macro avg       0.75      0.75      0.75     19833
weighted avg       0.75      0.75      0.75     19833

Predictions for new data:
[1]
